# Writing Agents
Test project for 224N/PWR

In [26]:
import os
from openai import OpenAI
from pydantic import BaseModel
from enum import Enum
from typing import List
import pprint
import pdfkit
import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
import create_doc



client = OpenAI()

MODEL = "gpt-4o-mini-2024-07-18"

In [27]:
paragraph_types = {
  "introduction": "Introduces the topic, provides background information, and states the thesis or main argument.",
  "expository": "Explains or summarizes a source, concept, or evidence to provide context or background for the argument.",
  "argumentative": "Presents a single point or claim in support of the thesis, supported by evidence and analysis.",
  "comparative": "Explores similarities or differences between two sources, arguments, or viewpoints to build the case.",
  "synthesizing": "Combines and connects multiple sources or arguments to show relationships and strengthen the thesis.",
  "counterargument": "Acknowledges and addresses opposing viewpoints or potential criticisms of the argument.",
  "transitional": "Facilitates smooth movement between major sections or arguments by summarizing and introducing connections.",
  "analytical": "Delves into the meaning, implications, or significance of a piece of evidence or a source.",
  "evaluative": "Critiques or evaluates a source or perspective to assess its strengths, weaknesses, or credibility.",
  "conclusion": "Summarizes main points, restates the thesis, and provides a call to action or final thought."
}


In [28]:
system_prompt = """
You are an expert author tasked with crafting a high-quality argumentative paper on a given topic, designed to resemble a compelling newspaper opinion piece. 
Your paper must follow a clear, logical structure and maintain a strong, engaging flow between paragraphs, avoiding redundancy while building a persuasive and cohesive argument. 

The paper must revolve around a well-defined thesis statement, which serves as the backbone of the argument. Every paragraph should explicitly or implicitly support the thesis, creating a unified and focused narrative.

### Instructions:
1. **Structure and Organization**:
   - Begin the paper by clearly defining the thesis statement, ensuring it is specific, debatable, and sets the tone for the argument.
   - The paper should include an outline with a compelling title and a sequence of well-structured paragraphs.
   - Each paragraph must serve a distinct purpose, explicitly or implicitly supporting the thesis and connecting seamlessly to the next for a natural progression of ideas.

2. **For Each Paragraph**:
   - Assign a sequential number.
   - Provide a descriptive name reflecting its role in the paper.
   - Specify the paragraph type using the provided taxonomy.
   - Include a concise, focused prompt designed to guide another LLM in generating the content for that paragraph.
   - Clearly articulate how the paragraph relates to and supports the thesis.

3. **Types of Paragraphs** (with descriptions):
   - **Introduction**: Grabs attention, introduces the topic, presents the thesis or main argument, and outlines the structure of the paper.
   - **Expository**: Explains key concepts, evidence, or background information essential for understanding the thesis.
   - **Argumentative**: Advances a specific claim supported by evidence and analysis that directly supports the thesis.
   - **Comparative**: Examines similarities or differences between two ideas, sources, or perspectives to highlight aspects that reinforce the thesis.
   - **Synthesizing**: Connects multiple sources, arguments, or ideas to build a unified perspective and strengthen the thesis.
   - **Counterargument**: Acknowledges opposing viewpoints, refutes them with reasoning and evidence, and reinforces the thesis.
   - **Transitional**: Bridges ideas or sections to ensure a smooth flow and logical progression while maintaining focus on the thesis.
   - **Analytical**: Delves into the implications, significance, or deeper meaning of evidence or a source in the context of the thesis.
   - **Evaluative**: Critiques a source, argument, or perspective, assessing its credibility, strengths, and weaknesses in relation to the thesis.
   - **Conclusion**: Summarizes the main arguments, reinforces the thesis, and provides a compelling closing statement or call to action.

4. **Writing Style**:
   - Use a concise, persuasive tone that mirrors the style of a top-tier opinion piece.
   - Avoid redundancy by ensuring each paragraph introduces new insights or ideas that contribute to and reinforce the thesis.
   - Prioritize logical flow between paragraphs to guide the reader smoothly through the argument while maintaining a consistent focus on the thesis.

5. **Output**:
   - The output should include a detailed outline of the paper with a dedicated section for the thesis statement.
   - Ensure each paragraph prompt is precise, relevant, and explicitly tied to the thesis.
   - Include space in the structured output to define the thesis statement and demonstrate how each paragraph supports it.
"""

user_prompt = "Write a paper supporting the use of facial recognition technology in airport security"


class ParagraphType(str, Enum):
  introduction = "introduction"
  expository = "expository"
  argumentative = "argumentative"
  comparative = "comparative"
  synthesizing = "synthesizing"
  counterargument = "counterargument"
  transitional = "transitional"
  analytical = "analytical"
  evaluative = "evaluative"
  conclusion = "conclusion"

class Paragraph(BaseModel):
  number: int
  name: str
  paragraphType: ParagraphType
  prompt: str
  
class PaperStructure(BaseModel):
  title: str
  thesis: str
  paragraphs: List[Paragraph]

# master thread
response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
        ],
    response_format=PaperStructure,
)

pprint.pprint(response.choices[0].message.parsed)

PaperStructure(title='Facial Recognition Technology: Enhancing Airport Security for a Safer Future', thesis='The implementation of facial recognition technology in airport security is essential for improving passenger safety, streamlining processes, and enhancing overall efficiency, while addressing privacy concerns through robust regulations and transparent practices.', paragraphs=[Paragraph(number=1, name='Engaging Introduction', paragraphType=<ParagraphType.introduction: 'introduction'>, prompt="Introduce the significance of airport security, highlighting current challenges and the evolving nature of threats. Present the thesis statement emphasizing the need for facial recognition technology in enhancing safety and efficiency at airports. Outline the structure of the paper, indicating the main points to be covered like safety benefits, efficiency improvements, and addressing privacy concerns.  Connect the importance of these elements to the overarching goal of safeguarding the publi

In [29]:
print (f"Title: {response.choices[0].message.parsed.title}")
print (f"Thesis: {response.choices[0].message.parsed.thesis}")
for paragraph in response.choices[0].message.parsed.paragraphs:
  print(f"{paragraph.number}.   {paragraph.name} ({paragraph.paragraphType.value})")
#   print(f"Prompt: {paragraph.prompt}")
#   print("\n")

Title: Facial Recognition Technology: Enhancing Airport Security for a Safer Future
Thesis: The implementation of facial recognition technology in airport security is essential for improving passenger safety, streamlining processes, and enhancing overall efficiency, while addressing privacy concerns through robust regulations and transparent practices.
1.   Engaging Introduction (introduction)
2.   Current Security Challenges (expository)
3.   Enhancing Passenger Safety (argumentative)
4.   Streamlining Airport Processes (argumentative)
5.   Addressing Privacy Concerns (counterargument)
6.   Conclusion: A Call to Action (conclusion)


### Writing the actual text

In [30]:
system_prompt = """

You are an expert writer tasked with crafting a single, high-quality paragraph for an argumentative paper.

You will be provided with the full structure of the paper, including the names and types of each paragraph in order, as well as a prompt defining the focus of the paragraph you are writing and the thesis of the paper.

Your writing must be concise, meaningful, and directly tied to the thesis while ensuring smooth transitions between paragraphs and sections.

### Instructions:

1. **Understand the Context**:

   - Review the full structure of the paper to understand the flow and relationships between paragraphs.
   - Identify the role of the paragraph you are crafting within the structure and how it connects to the previous and next paragraphs.
   - Use the provided prompt to craft a focused, purposeful paragraph that aligns with the thesis and contributes to the overall logical progression of the paper.

2. **Paragraph Types**:

   - **Introduction**: Hook the reader, introduce the topic, present the thesis, and briefly outline the paper’s key arguments. Ensure this paragraph establishes a strong foundation for the paper’s flow.
   - **Expository**: Provide essential context or explain key evidence directly related to the thesis. Connect the context to the prior argument and set up the next paragraph.
   - **Argumentative**: Present a strong, specific claim backed by evidence that directly supports the thesis. Conclude by preparing the reader for the next argument or evidence.
   - **Comparative**: Analyze similarities or differences to highlight aspects that strengthen the thesis. Smoothly connect comparisons to prior and forthcoming paragraphs.
   - **Synthesizing**: Combine ideas or sources to form a cohesive argument that advances the thesis. Tie synthesized ideas to the preceding discussion and suggest implications for the next section.
   - **Counterargument**: Address and refute opposing views with clear evidence and reasoning. Transition smoothly from prior points and guide the reader back to the thesis.
   - **Transitional**: Connect ideas or sections to ensure smooth, logical progression while maintaining focus on the thesis. Serve as a bridge that reinforces continuity and introduces the next section.
   - **Analytical**: Explore the deeper implications or significance of evidence in relation to the thesis. Link implications to prior evidence and analysis and set up subsequent arguments.
   - **Evaluative**: Critique a source or argument, focusing on its relevance and impact on the thesis. Ensure the critique builds on prior evidence and analysis and transitions to the next key point.
   - **Conclusion**: Summarize key points, restate the thesis, and provide a strong closing insight or call to action. The final sentence should unify the discussion and leave a lasting impression.

3. **Writing Style**:

   - Use clear, direct language that conveys meaningful content without unnecessary words.
   - Avoid redundancy and focus on presenting new insights or advancing the argument.
   - Maintain a logical flow that ties each paragraph to the thesis and ensures smooth progression between ideas and sections.

4. **Paragraph Structure**:

   - Begin with a topic sentence that establishes the paragraph’s main idea and links it to the previous paragraph.
   - Support the idea with concise evidence, analysis, or reasoning.
   - End with a sentence that reinforces the thesis and transitions logically to the next section.

5. **Additional Guidance for Full Paper Structure**:

   - Refer to the names and types of each paragraph to understand their individual roles and how they contribute to the overall argument.
   - Ensure each paragraph builds on the ideas established in previous paragraphs and sets up the next for a cohesive narrative.
   - Use transitional phrases and logical connections to maintain smooth and seamless progression.
   - If you can reasonably assume that an abbreviation has been defined in a previous paragraph or in the thesis, you should not redefine it and can use it as needed.

6. **Output**:

   - Write a single paragraph of 150–250 words unless specified otherwise.
   - Ensure the paragraph is polished, precise, and ready to be part of the larger argument.
   - Explicitly address the prompt, connect to the thesis in a meaningful way, and ensure smooth transitions from and to other paragraphs.

"""




responses = []
thesis = response.choices[0].message.parsed.thesis

for paragraph in response.choices[0].message.parsed.paragraphs:
    user_prompt = "Name: <name>" + paragraph.name + "</name>\n Type: <type>" + paragraph.paragraphType.value + "</type>\n Prompt: <prompt>" + paragraph.prompt + "</prompt>\n Thesis: <thesis>" + thesis + "</thesis>\n Paragraph Structure: <structure>" + "".join([f"{i}. {paragraph.name} ({paragraph.paragraphType.value})\n" for i, paragraph in enumerate(response.choices[0].message.parsed.paragraphs)]) + "</structure>"

    r = client.beta.chat.completions.parse(
         model=MODEL,
         messages=[
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": user_prompt}
               ]
      )
    responses.append(r)

print(responses[0])

ParsedChatCompletion[NoneType](id='chatcmpl-ApMqS6qOVj58mGDHMHapjBUOoMeNB', choices=[ParsedChoice[NoneType](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NoneType](content="In an age where global travel is increasingly vital yet fraught with security challenges, the significance of airport security has never been more pronounced. Recent incidents, such as the near-miss scenarios involving unauthorized access to restricted areas or threats posed by unchecked travelers, underscore the escalating nature of risks faced by airports today. As threats evolve, so too must our approaches to security, necessitating the integration of advanced technological solutions. Chief among these is facial recognition technology, which not only offers the potential for enhanced safety but also streamlines the often cumbersome processes encountered by passengers. This paper will explore the multifaceted benefits of implementing facial recognition systems at airports, inclu

In [31]:
paragraphs = [r.choices[0].message.content for r in responses]
pprint.pprint(paragraphs[3])

('Facial recognition technology has proven instrumental in streamlining '
 'airport processes, effectively reducing wait times and enhancing the overall '
 'passenger experience. For instance, a case study at the Hartsfield-Jackson '
 'Atlanta International Airport demonstrated that the introduction of facial '
 'recognition systems decreased boarding times by up to 30%, allowing for '
 'smoother transitions from check-in to gate. Similarly, airports like London '
 'Heathrow have reported a significant increase in passenger flow through '
 'security checks, with processing speeds improving by 50%, thanks to '
 'automated facial recognition. These efficiency gains not only alleviate '
 'bottlenecks during peak travel periods but also contribute to heightened '
 'customer satisfaction levels, as travelers benefit from expedited screening '
 'without compromising safety. In fact, surveys conducted post-implementation '
 'reveal that 85% of passengers felt more secure knowing that their id

In [32]:
# Sample data (replace with your own references)
title = response.choices[0].message.parsed.title

create_doc.create_document(paragraphs, thesis, title)

Created doc with ID: 19nuh2xK3xbj4PVmyE_i-Lj8GXXEfYxCFhS7FzkGuO0E
Document created successfully! View it at: https://docs.google.com/document/d/19nuh2xK3xbj4PVmyE_i-Lj8GXXEfYxCFhS7FzkGuO0E/edit
